=========

Merging with the right-hand expression:

$$
=\frac
  {\alpha^{K^+}}
  {\prod_{h=1}^{2^N-1}K_h!}
\left(
\frac
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)}
  {\Gamma(N + 1)}
  \right)^{K}
\prod_{k=1}^{K^+}
\left(
\frac
  {\Gamma(N + 1)(m_k - 1)!(N-m_k)!}
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)\,N!}
  \right)
$$

$$
=\frac
  {\alpha^{K^+}}
  {\prod_{h=1}^{2^N-1}K_h!}
\left(
\frac
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)}
  {\Gamma(N + 1)}
  \right)^{K}
\prod_{k=1}^{K^+}
\left(
\frac
  {(m_k - 1)!(N-m_k)!}
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)}
  \right)
$$

Looking just at the right hand expression:

$$
\prod_{k=1}^{K^+}
\left(
\frac
  {(m_k - 1)!(N-m_k)!}
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)}
  \right)
$$

Since $K^+$ is finite (since expected number of total non-zeros is upper-bounded by $N\alpha$, so $K^+ \le N\alpha$), so we can remove the terms in $\alpha/K$, which will tend to 0:

$$
\prod_{k=1}^{K^+}
\left(
\frac
  {(m_k - 1)!(N-m_k)!}
  {\Gamma(1) \Gamma(N+1)}
  \right)
$$

$$
=\prod_{k=1}^{K^+}
\left(
\frac
  {(m_k - 1)!(N-m_k)!}
  {N!}
  \right)
$$

... which matches the tutorial (though I'm not sure how clean my working is here...)

For the middle term, which will becomes $\exp(-\alpha\sum_{j=1}^N \frac{1}{j}$, we have:

$$
\left(
\frac
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)}
  {\Gamma(N + 1)}
  \right)^{K}
$$

... and we want it to become:

$$
\left(
\frac{N!}
  {\prod_{j=1}^N(j + \frac{\alpha}{K})}
\right)^K
$$


========

$$
=\frac
  {\alpha^{K^+}}
  {\prod_{h=1}^{2^N-1}K_h!}
\left(
\frac
  {\Gamma(\alpha/K+1) \Gamma(N+\alpha/K+1)}
  {\Gamma(N + 1)}
  \right)^{K}
\prod_{k=1}^{K^+}
\left(
\frac
  {(m_k - 1)!(N-m_k)!}
  {\prod_{j=0}^{N-1}(j + \alpha/K)\, \Gamma(\alpha/K+1) \Gamma(\alpha/K+1)}
  \right)
$$

Looking just at the right hand expression:

$$
\prod_{k=1}^{K^+}
\left(
\frac
  {(m_k - 1)!(N-m_k)!}
  {\prod_{j=0}^{N-1}(j + \alpha/K)\, \Gamma(\alpha/K+1) \Gamma(\alpha/K+1)}
  \right)
$$

Since $K+$ is finite, so $\Gamma(\alpha/K+1) \to 1$, and can be removed:

$$
\prod_{k=1}^{K^+}
\left(
\frac
  {(m_k - 1)!(N-m_k)!}
  {\prod_{j=0}^{N-1}(j + \alpha/K)}
  \right)
$$


Let's start by expanding out $\Gamma(N + \alpha/K + 1)$ to $\Gamma(\alpha/K + 1) \prod_{j=1}^{N}(\alpha/K + j)$, since that somewhat matches the numerator of the term we want.  So we get:

$$
\left(
\frac
  {\Gamma(\alpha/K+1) \Gamma(\alpha/K + 1) \prod_{j=1}^{N}(\alpha/K + j)}
  {\Gamma(N + 1)}
  \right)^{K}
$$
